In [ ]:
import pandas as pd
from cleaning import create_demand
import pyomo.environ as pyo

# Data import

In [ ]:
nodes = pd.read_pickle("../data/original/nodes.pkl")
channels = pd.read_pickle("../data/original/channels.pkl")

In [1]:
nodes.to_csv("../data/original/nodes.csv")
channels.to_csv("../data/original/channels.csv")

NameError: name 'nodes' is not defined

## Modeling

Change the logic of CHANNELS set construction by considering the pair of nodes as the edge identifier.
In this way I also have to deal with:
1. multiedges, aka multiple edges between two nodes. I can not consider those for simplicity
2. directed relationship. I need to understand if flipping the channels - as done before - can be reasonable. I also need to determine if the flipped pair of nodes is considered a valid index by pyomo.

In [ ]:
#channels.reset_index(inplace=True)

In [ ]:
#channels.set_index(["node1_pub", "node2_pub"], inplace=True)

In [ ]:
#channels.sort_index(inplace=True)

In [ ]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
#model.CHANNELS = pyo.Set(initialize=channels.index)
model.CHANNELS = pyo.Set(initialize=[(channels.loc[i, "node1_pub"], channels.loc[i, "node2_pub"]) for i in channels.index])



In [ ]:
#model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals)

In [1]:
###def objective_function(model: pyo.ConcreteModel):
#    return sum(channels.loc[k, "base_fee"] * model.x[k] for k in model.CHANNELS) + sum(channels.loc[i, "rate_fee"] * model.a[i] for i in model.CHANNELS)

def objective_function(model: pyo.ConcreteModel):
    return sum(channels.loc[(channels["node1_pub"]==i[0]) & (channels["node2_pub"]==i[1]), "rate_fee"] * model.a[i] for i in model.CHANNELS)

model.totalCost = pyo.Objective(expr=objective_function(model), sense=pyo.minimize)

NameError: name 'pyo' is not defined

In [ ]:
#for i in model.CHANNELS:
#    print(sum(channels.loc[(channels["node1_pub"]==i[0]) & (channels["node2_pub"]==i[1]), "rate_fee"] * model.a[i]))


### Constraints

#### Capacity constraint

$$amount_{i,j} \le capacity_{i,j} \times x_{i,j} \text{ } \forall (i,j) \in E$$


In [ ]:
def capacity_constraint(model: pyo.ConcreteModel, a):
    return model.a[a] <= channels.loc[a, "capacity"] # * model.x[a]

model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

#### Flow balance constraint

$$\sum_{(s,i) \in E} amount_{si} - \sum_{(i,t) \in E} amount_{it} = b_i \text{ } \forall i \in V$$


In [ ]:
nodes = create_demand(nodes)

In [ ]:

def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
    return sum(model.a[a] for a in nodes.loc[n, 'incoming_channels']) - sum(model.a[a] for a in nodes.loc[n, 'outgoing_channels']) == nodes.loc[n, "demand"]

model.FlowBalanceConstraint = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")


## Solving the model

In [ ]:
opt = pyo.SolverFactory('cbc')
results = opt.solve(model, tee=True)

if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print('\nOptimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.feasible:
    print('\nFeasible but not proven optimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
    raise Exception("The model is infeasible")
else:
    print('\nSolver Status: ',  results.solver.status)
    raise Exception(results.solver.status)

print('\nObject function value = ', model.Objective())


In [ ]:
ris = []
for i,v in model.a.extract_values().items():
    if v != 0:
      ris.append((i, v))
      
print(ris)